***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [ ]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

In [ ]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
spark

In [ ]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'bucket_wiki1' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [ ]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs_title = parquetFile.select("title", "id").rdd
doc_text_pairs_anchor = parquetFile.select("anchor_text", "id").rdd
doc_text_pairs_body = parquetFile.select("text", "id").rdd


In [ ]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

In [ ]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
from inverted_index_gcp import InvertedIndex

In [ ]:
import math
import statistics
bucket_name = 'ir_resources'
stemmer = PorterStemmer()
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
    return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE
def word_count(text, id):
    
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    map = {}
    for w in tokens:
        if w in all_stopwords:
            continue
        
        map[w] = map.get(w, 0) + 1
    return [(k,(id,v)) for k,v in map.items()]

def word_count_stemming(text, id):
    
    tokens = [stemmer.stem(token.group()) for token in RE_WORD.finditer(text.lower())]
    map = {}
    for w in tokens:
        if w in all_stopwords:
            continue
        
        map[w] = map.get(w, 0) + 1
    return [(k,(id,v)) for k,v in map.items()]


def reduce_word_counts(unsorted_pl):
    return sorted(unsorted_pl)

def calculate_df(postings):
    return postings.map(lambda x:(x[0],len(x[1])))

def partition_postings_and_write(postings,base_dir):   
    posting_lst_with_bucket = postings.map(lambda x:((token2bucket_id(x[0]),x)))
    posting_lst_group_by_bucket = posting_lst_with_bucket.groupByKey()
    return posting_lst_group_by_bucket.map(lambda x :InvertedIndex.write_a_posting_list(x,bucket_name,base_dir))
def calculate_dl(postings):
    return postings.map(lambda x:x[1]).flatMap(lambda x:x).reduceByKey(lambda x,y :x+y)
def calculate_tt(postings):
    return postings.map(lambda x: (x[0], np.sum([y[1] for y in x[1]])))
def calculate_nf(postings):
    return postings.map(lambda x: x[1]).flatMap(lambda x:x).map(lambda x:(x[0],x[1]**2)).groupByKey().mapValues(list).map(lambda x:(x[0],1/math.sqrt(np.sum(x[1]))))
def calculate_AVGDL(DL):
    return statistics.mean(DL.values())


# title inverted index

In [ ]:
word_counts_title = doc_text_pairs_title.flatMap(lambda x: word_count(x[0], x[1]))

In [ ]:
postings_title = word_counts_title.groupByKey().mapValues(reduce_word_counts)

In [ ]:
w2df_title = calculate_df(postings_title).collectAsMap()

In [ ]:
tt_title = calculate_tt(postings_title).collectAsMap()

In [ ]:
DL_title = calculate_dl(postings_title).collectAsMap()

In [ ]:
nf_title = calculate_nf(postings_title).collectAsMap()

In [ ]:
_ = partition_postings_and_write(postings_title,"title_index").collect()

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs_title = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='title_index'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs_title = pickle.load(f)
        for k, v in posting_locs_title.items():
            super_posting_locs_title[k].extend(v)

Putting it all together

In [ ]:
# Create inverted index instance
inverted_title = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_title.posting_locs = super_posting_locs_title
# Add the token - df dictionary to the inverted index
inverted_title.df = w2df_title
inverted_title.DL = DL_title
inverted_title.term_total = tt_title
inverted_title.nf = nf_title
inverted_title.AVGDL = calculate_AVGDL(DL_title)

# write the global stats out
inverted_title.write_index('.', 'title_index')

# upload to gs
index_src = "title_index.pkl"
index_dst = f'gs://{bucket_name}/title_index/{index_src}'
!gsutil cp $index_src $index_dst

In [ ]:
!gsutil ls -lh $index_dst

# body inverted index

In [ ]:
word_counts_body = doc_text_pairs_body.flatMap(lambda x: word_count(x[0], x[1]))

In [ ]:
postings_body = word_counts_body.groupByKey().mapValues(reduce_word_counts)
postings_body_filtered = postings_body.filter(lambda x: len(x[1])>50)

In [ ]:
w2df_body = calculate_df(postings_body_filtered).collectAsMap()

In [ ]:
tt_body = calculate_tt(postings_body_filtered).collectAsMap()

In [ ]:
DL_body = calculate_dl(postings_body_filtered).collectAsMap()

In [ ]:
nf_body = calculate_nf(postings_body_filtered).collectAsMap()

In [ ]:
_ = partition_postings_and_write(postings_body_filtered,"body_index").collect()

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs_body = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='body_index'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs_body = pickle.load(f)
        for k, v in posting_locs_body.items():
            super_posting_locs_body[k].extend(v)

In [ ]:
# Create inverted index instance
inverted_body = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_body.posting_locs = super_posting_locs_body
# Add the token - df dictionary to the inverted index
inverted_body.df = w2df_body
inverted_body.DL = DL_body
inverted_body.term_total = tt_body
inverted_body.nf = nf_body
inverted_body.AVGDL = calculate_AVGDL(DL_body)

# write the global stats out
inverted_body.write_index('.', 'body_index')

# upload to gs
index_src = "body_index.pkl"
index_dst = f'gs://{bucket_name}/body_index/{index_src}'
!gsutil cp $index_src $index_dst

In [ ]:
!gsutil ls -lh $index_dst

# anchor inverted index

In [ ]:
doc_text_pairs_anchor = parquetFile.select("id","anchor_text").rdd
postings_doc_lst_titles = doc_text_pairs_anchor.map(lambda x:[(y[0], y[1]) for y in x[1]])
postings_doc_lst_titles = postings_doc_lst_titles.flatMap(lambda x:[word_count(text, id) for id,text in x])
postings_doc_lst_titles = postings_doc_lst_titles.flatMap(lambda x : x)
postings_doc_lst_titles =postings_doc_lst_titles.groupByKey().mapValues(list).map(lambda x: (x[0],list(set([y[0] for y in x[1]]))))
postings_doc_lst_titles = postings_doc_lst_titles.map(lambda x:(x[0],[(y,1) for y in x[1]]))
postings_doc_lst_titles = postings_doc_lst_titles.map(lambda x : (x[0],reduce_word_counts(x[1])))

In [ ]:
w2df_anchor = calculate_df(postings_doc_lst_titles).collectAsMap()

In [ ]:
_ = partition_postings_and_write(postings_doc_lst_titles,"anchor_index").collect()

In [ ]:
super_posting_locs_anchor = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='anchor_index'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs_anchor = pickle.load(f)
        for k, v in posting_locs_anchor.items():
            super_posting_locs_anchor[k].extend(v)

In [ ]:
# Create inverted index instance
inverted_anchor = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_anchor.posting_locs = super_posting_locs_anchor
inverted_anchor.df = w2df_anchor
# write the global stats out
inverted_anchor.write_index('.', 'anchor_index')

# upload to gs
index_src = "anchor_index.pkl"
index_dst = f'gs://{bucket_name}/anchor_index/{index_src}'

In [ ]:
!gsutil cp $index_src $index_dst

In [ ]:
!gsutil cp $index_src $index_dst

# title to doc

In [ ]:
docid_title_dict = parquetFile.select("id","title").rdd.collectAsMap()
with open("docid_title_dict.pkl", 'wb') as f:
    pickle.dump(docid_title_dict, f)

In [ ]:
index_src = "docid_title_dict.pkl"
index_dst = f'gs://{bucket_name}/{index_src}'

In [ ]:
!gsutil cp $index_src $index_dst

# body index doc-(word-td)

In [ ]:
NUM_BUCKETS = 124
def token2bucket_id(doc_id):
    return doc_id % NUM_BUCKETS

def word_count_doc(text, id):
    
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    map = {}
    for w in tokens:
        if w in all_stopwords:
            continue
        if len(w) > 12:
            continue
        map[w] = map.get(w, 0) + 1
    return [(k,(id,v)) for k,v in map.items()]

def partition_postings_and_write_doc(postings,base_dir):
    posting_lst_with_bucket = postings.map(lambda x:((token2bucket_id(x[0]),x)))
    posting_lst_group_by_bucket = posting_lst_with_bucket.groupByKey()
    return posting_lst_group_by_bucket.map(lambda x :InvertedIndex.write_a_posting_list_doc(x,bucket_name,base_dir))

In [ ]:
doc_text_pairs_body = parquetFile.select("text", "id").rdd

In [ ]:
word_counts_body = doc_text_pairs_body.flatMap(lambda x: word_count_doc(x[0], x[1]))
postings_body = word_counts_body.groupByKey().mapValues(reduce_word_counts)
postings_body_filtered = postings_body.filter(lambda x: len(x[1])>50)


In [ ]:
postings_body_doc_term = postings_body_filtered.flatMap(lambda x: [(doc_id,(x[0],tf)) for doc_id,tf  in x[1]]).groupByKey().mapValues(list)

In [ ]:
postings_body_doc_term_sort = postings_body_doc_term.map(lambda x:(x[0],sorted(x[1],key= lambda y:y[1],reverse=True)))


In [ ]:
docs2df_body = calculate_df(postings_body_doc_term_sort).collectAsMap()

In [ ]:
_ = partition_postings_and_write_doc(postings_body_doc_term_sort,"body_docs_index").collect()

In [ ]:
super_posting_locs_body_doc = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='body_docs_index'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs_body_doc = pickle.load(f)
        for k, v in posting_locs_body_doc.items():
            super_posting_locs_body_doc[k].extend(v)

In [ ]:
# Create inverted index instance
inverted_body_doc = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_body_doc.posting_locs = super_posting_locs_body_doc
# Add the token - df dictionary to the inverted index
inverted_body_doc.df = docs2df_body

# write the global stats out
inverted_body_doc.write_index('.', 'body_docs_index')

# upload to gs
index_src = "body_docs_index.pkl"
index_dst = f'gs://{bucket_name}/body_docs_index/{index_src}'
!gsutil cp $index_src $index_dst

# title inverted index_stemming

In [ ]:
word_counts_title_stemming = doc_text_pairs_title.flatMap(lambda x: word_count_stemming(x[0], x[1]))

In [ ]:
postings_title_stemming = word_counts_title_stemming.groupByKey().mapValues(reduce_word_counts)

In [ ]:
w2df_title_stemming = calculate_df(postings_title_stemming).collectAsMap()

In [ ]:
tt_title_stemming = calculate_tt(postings_title_stemming).collectAsMap()

In [ ]:
DL_title_stemming = calculate_dl(postings_title_stemming).collectAsMap()

In [ ]:
nf_title_stemming = calculate_nf(postings_title_stemming).collectAsMap()

In [ ]:
_ = partition_postings_and_write(postings_title_stemming,"title_index_stemming").collect()

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs_title_stemming = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='title_index_stemming'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs_title_stemming = pickle.load(f)
        for k, v in posting_locs_title_stemming.items():
            super_posting_locs_title_stemming[k].extend(v)

Putting it all together

In [ ]:
# Create inverted index instance
inverted_title_stemming = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_title_stemming.posting_locs = super_posting_locs_title_stemming
# Add the token - df dictionary to the inverted index
inverted_title_stemming.df = w2df_title_stemming
inverted_title_stemming.DL = DL_title_stemming
inverted_title_stemming.term_total = tt_title_stemming
inverted_title_stemming.nf = nf_title_stemming
inverted_title_stemming.AVGDL = calculate_AVGDL(DL_title_stemming)

# write the global stats out
inverted_title_stemming.write_index('.', 'title_index_stemming')

# upload to gs
index_src = "title_index_stemming.pkl"
index_dst = f'gs://{bucket_name}/title_index_stemming/{index_src}'
!gsutil cp $index_src $index_dst

In [ ]:
!gsutil ls -lh $index_dst

# body inverted index_stemming

In [ ]:
word_counts_body_stemming = doc_text_pairs_body.flatMap(lambda x: word_count_stemming(x[0], x[1]))

In [ ]:
postings_body_stemming = word_counts_body_stemming.groupByKey().mapValues(reduce_word_counts)
postings_body_filtered_stemming = postings_body_stemming.filter(lambda x: len(x[1])>50)

In [ ]:
w2df_body_stemming = calculate_df(postings_body_filtered_stemming).collectAsMap()

In [ ]:
tt_body_stemming = calculate_tt(postings_body_filtered_stemming).collectAsMap()

In [ ]:
DL_body_stemming = calculate_dl(postings_body_filtered_stemming).collectAsMap()

In [ ]:
nf_body_stemming = calculate_nf(postings_body_filtered_stemming).collectAsMap()

In [ ]:
_ = partition_postings_and_write(postings_body_filtered_stemming,"body_index_stemming").collect()

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs_body_stemming = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='body_index_stemming'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs_body_stemming = pickle.load(f)
        for k, v in posting_locs_body_stemming.items():
            super_posting_locs_body_stemming[k].extend(v)

In [ ]:
# Create inverted index instance
inverted_body_stemming = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_body_stemming.posting_locs = super_posting_locs_body_stemming
# Add the token - df dictionary to the inverted index
inverted_body_stemming.df = w2df_body_stemming
inverted_body_stemming.DL = DL_body_stemming
inverted_body_stemming.term_total = tt_body_stemming
inverted_body_stemming.nf = nf_body_stemming
inverted_body_stemming.AVGDL = calculate_AVGDL(DL_body_stemming)

# write the global stats out
inverted_body_stemming.write_index('.', 'body_index_stemming')

# upload to gs
index_src = "body_index_stemming.pkl"
index_dst = f'gs://{bucket_name}/body_index_stemming/{index_src}'
!gsutil cp $index_src $index_dst

In [ ]:
!gsutil ls -lh $index_dst